In [1]:
from deep_learning_features_audio import *
from deep_learning_dict_api import AudioAnalysisAPI
from deep_learning_dict_datasets import Datasets
import evaluate
from evaluate import load
import pandas as pd
from pathlib import Path
from IPython.display import display, HTML
from torchmetrics import ScaleInvariantSignalNoiseRatio, ScaleInvariantSignalDistortionRatio, SignalNoiseRatio, SignalDistortionRatio, PermutationInvariantTraining
from torchmetrics.audio.pesq import PerceptualEvaluationSpeechQuality
from torchmetrics.functional.audio import signal_distortion_ratio
from torchmetrics.audio.stoi import ShortTimeObjectiveIntelligibility

import numpy as np


/home/valeriopuglisi/.conda/envs/DLAABE/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# When running this tutorial in Google Colab, install the required packages
# with the following.
# !pip install torchaudio librosa boto3

import torch
import torchaudio
import torchaudio.functional as TAF
import torchaudio.transforms as T

print(torch.__version__)
print(torchaudio.__version__)

1.11.0+cu102
0.11.0+cu102


In [4]:
def speech_separation_evaluate_metric_with_model_on_librimix_2_channels(model, dataset, metrics, n_test):
    task = "Speech Separation"
    test_mix_both_file = "test_mix_both_file"
    total_si_snr = torch.zeros(0)
    total_si_sdr = torch.zeros(0)
    total_snr = torch.zeros(0)
    total_sdr = torch.zeros(0)
    total_nb_pesq = torch.zeros(0)
    total_wb_pesq = torch.zeros(0)
    total_pit = torch.zeros(0)
    total_stoi= torch.zeros(0)

    result = {}
    # os.walk(dataset_path)
    print("===================================================>  Dataset: {}".format(dataset))
    # print(Datasets[task])
    print("Datasets[task][dataset] : {}".format(Datasets[task][dataset][test_mix_both_file]))
    test_table = pd.read_table(Datasets[task][dataset][test_mix_both_file], sep=",")
    # display(test_table)
    # cols = test_table.iloc[:,1:4]
    print("len(test_table): {}".format(len(test_table)))
    # display(cols)
    for i, row in enumerate(test_table.iterrows()):
        preds = torch.zeros(0) 
        target = torch.zeros(0)
        print("====> Benchmarking: {}/{}   tot {}".format(i+1, n_test, test_table.shape[0]))
        # print(i, row[1]['mixture_path'], , , row[1]['noise_path'])
        mixture_path = row[1]['mixture_path']
        source_1_path = row[1]['source_1_path']
        source_2_path = row[1]['source_2_path']
        noise_path = row[1]['noise_path']
        source_1_path_prediction, source_2_path_prediction = AudioAnalysisAPI[model]['function'](audiofile_path=mixture_path)
        mixture_waveform, mixture_sample_rate = torchaudio.load(mixture_path)
        source_1_waveform, source_1_sample_rate = torchaudio.load(source_1_path)
        source_2_waveform, source_2_sample_rate = torchaudio.load(source_2_path)
        source_1_prediction_waveform, source_1_prediction_sample_rate = torchaudio.load(source_1_path_prediction)
        source_2_prediction_waveform, source_2_prediction_sample_rate = torchaudio.load(source_2_path_prediction)
        resample_rate_src_1 = max(source_1_sample_rate, source_1_prediction_sample_rate)
        resample_rate_src_2 = max(source_2_sample_rate, source_2_prediction_sample_rate)
        resample_rate = max(resample_rate_src_1, resample_rate_src_2) 
        # resampling to maximum frequency because is not sure that a model works with the original frequency
        source_1_waveform_resampled = TAF.resample(source_1_waveform, source_1_sample_rate, resample_rate)
        source_2_waveform_resampled = TAF.resample(source_2_waveform, source_2_sample_rate, resample_rate)
        source_1_prediction_waveform_resampled = TAF.resample(source_1_prediction_waveform, source_1_prediction_sample_rate, resample_rate)
        source_2_prediction_waveform_resampled = TAF.resample(source_2_prediction_waveform, source_2_prediction_sample_rate, resample_rate)
        preds = torch.cat((preds, source_1_prediction_waveform_resampled), 0)
        preds = torch.cat((preds, source_2_prediction_waveform_resampled), 0)
        target = torch.cat((target, source_1_waveform_resampled), 0)
        target = torch.cat((target, source_2_waveform_resampled), 0)
        # print("==> Mixture : {}".format(mixture_path))
        # print("==> source_1 : {}".format(source_1_path))
        # print("==> source_2 : {}".format(source_2_path))
        # print("==> source_1_prediction : {}".format(source_1_path_prediction))
        # print("==> source_2_prediction : {}".format(source_2_path_prediction))
        """
        print("mixture_waveform.shape:{} - mixture_sample_rate:{}".format(mixture_waveform.shape, mixture_sample_rate))
        print("source_1_waveform.shape:{}, source_1_sample_rate:{}".format(source_1_waveform.shape, source_1_sample_rate))
        print("source_2_waveform.shape:{}, source_2_sample_rate:{}".format(source_2_waveform.shape, source_2_sample_rate))
        print("source_1_prediction_waveform.shape : {}, source_1_prediction_sample_rate: {}".format(source_1_prediction_waveform.shape, source_1_prediction_sample_rate))
        print("source_2_prediction_waveform.shape : {}, source_2_prediction_sample_rate: {}".format(source_2_prediction_waveform.shape, source_2_prediction_sample_rate))
        print("resample_rate_src_1: {}".format(resample_rate_src_1))
        print("resample_rate_src_2: {}".format(resample_rate_src_2))
        print("resample_rate: {}".format(resample_rate))
        print("source_1_waveform_resampled.shape: {}".format(source_1_waveform_resampled.shape))
        print("source_2_waveform_resampled.shape: {}".format(source_2_waveform_resampled.shape))
        print("source_1_prediction_waveform_resampled.shape : {}".format(source_1_prediction_waveform_resampled.shape))
        print("source_2_prediction_waveform_resampled.shape : {}".format(source_2_prediction_waveform_resampled.shape))
        print("preds.shape {}, target.shape:{}".format(preds.shape, target.shape))
        print("preds {}, target:{}".format(preds, target))
        """
        for metric in metrics:
        #print("Calculating metric:", metric)
            if metric == "si-snr":
                si_snr = ScaleInvariantSignalNoiseRatio()
                si_snr_result = si_snr(preds, target)
                si_snr_result = torch.reshape(si_snr_result, (1, 1))
                total_si_snr = torch.cat((total_si_snr, si_snr_result))

            if metric == "si-sdr":
                si_sdr = ScaleInvariantSignalDistortionRatio()
                si_sdr_result = si_sdr(preds, target)
                si_sdr_result = torch.reshape(si_sdr_result, (1, 1))
                total_si_sdr = torch.cat((total_si_sdr, si_sdr_result))                 

            if metric == "snr":
                snr = SignalNoiseRatio()
                snr_result = snr(preds, target)
                snr_result = torch.reshape(snr_result, (1, 1))
                total_snr = torch.cat((total_snr, snr_result))
                

            if metric == "sdr":
                sdr = SignalDistortionRatio()
                sdr_result = sdr(preds, target)
                sdr_result = torch.reshape(sdr_result, (1, 1))
                total_sdr = torch.cat((total_sdr, sdr_result))

            if metric == "pesq":
                nb_pesq = PerceptualEvaluationSpeechQuality(resample_rate, 'nb')
                nb_pesq_result = nb_pesq(preds, target)
                nb_pesq_result = torch.reshape(nb_pesq_result, (1, 1))
                total_nb_pesq = torch.cat((total_nb_pesq, nb_pesq_result))

                if resample_rate > 8000: 
                    wb_pesq = PerceptualEvaluationSpeechQuality(resample_rate, 'wb')
                    wb_pesq_result = wb_pesq(preds, target)
                    wb_pesq_result = torch.reshape(wb_pesq_result, (1, 1))
                    total_wb_pesq = torch.cat((total_wb_pesq, wb_pesq_result))

            if metric == "pit":
                pit = PermutationInvariantTraining(signal_distortion_ratio, 'max')
                pit_result = pit(preds, target)
                pit_result = torch.reshape(pit_result, (1, 1))
                total_pit = torch.cat((total_pit, pit_result))
                

            if metric == "stoi":
                stoi_src = ShortTimeObjectiveIntelligibility(resample_rate, False)
                stoi_result = stoi_src(preds, target)
                stoi_result = torch.reshape(stoi_result, (1, 1))
                total_stoi = torch.cat((total_stoi, stoi_result))
                
        if i == n_test - 1: 
            break

    
    total_si_snr = torch.sum(total_si_snr)/n_test
    total_si_sdr = torch.sum(total_si_sdr)/n_test
    total_snr = torch.sum(total_snr)/n_test
    total_sdr = torch.sum(total_sdr)/n_test
    total_pit = torch.sum(total_pit)/n_test
    total_wb_pesq = torch.sum(total_wb_pesq)/n_test
    total_nb_pesq = torch.sum(total_nb_pesq)/n_test
    total_stoi = torch.sum(total_stoi)/n_test

    print("============================================================================================")
    print("total_si_snr:{}".format(total_si_snr)) 
    print("total_si_sdr:{}".format(total_si_sdr)) 
    print("total_snr:{}".format(total_snr)) 
    print("total_sdr:{}".format(total_sdr)) 
    print("total_pit:{}".format(total_pit)) 
    print("total_wb_pesq:{}".format(total_wb_pesq))  
    print("total_nb_pesq:{}".format(total_nb_pesq))  
    print("total_stoi:{}".format(total_stoi))  

    params = {"model": model, "dataset": dataset, "n_test": n_test}
    evaluate.save(path_or_file="./results/", **result, **params)
    
    return result

speech_separation_2_channels_models = ['/api/audioseparation/speech_separation_sepformer_wsj02mix']
speech_separation_dataset = ["Libri2Mix8kMin", "Libri2Mix8kMax", "Libri2Mix16kMin", "Libri2Mix16kMax"]
metrics = ["si-snr", "si-sdr", "sdr", "snr", "pesq", "pit", "stoi"]
n_test = 3000
speech_separation_evaluate_metric_with_model_on_librimix_2_channels(
    speech_separation_2_channels_models[0], 
    speech_separation_dataset[0],
    [metrics[0], metrics[1], metrics[2], metrics[3], metrics[4], metrics[6]], 
    n_test=n_test )
#speech_separation_evaluate_metric_with_model_on_librimix_2_channels(speech_separation_2_channels_models[0], speech_separation_dataset[1], [metrics[0]], n_test=n_test )
# speech_separation_evaluate_metric_with_model_on_librimix_2_channels(speech_separation_2_channels_models[0], speech_separation_dataset[2], [metrics[0]], n_test=n_test )
# speech_separation_evaluate_metric_with_model_on_librimix_2_channels(speech_separation_2_channels_models[0], speech_separation_dataset[3], [metrics[0]], n_test=n_test )




===================================================>  Dataset: Libri2Mix8kMin
Datasets[task][dataset] : /storage/data_8T/datasets/audio/LibriMix/Libri2Mix/wav8k/min/metadata/mixture_test_mix_both.csv
len(test_table): 3000
====> Benchmarking: 1/3000   tot 3000


/home/valeriopuglisi/.conda/envs/DLAABE/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (ScaleInvariantSignalNoiseRatio). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_full_state_property`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)
/home/valeriopuglisi/.conda/envs/DLAABE/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36:

====> Benchmarking: 2/3000   tot 3000
====> Benchmarking: 3/3000   tot 3000
====> Benchmarking: 4/3000   tot 3000
====> Benchmarking: 5/3000   tot 3000
====> Benchmarking: 6/3000   tot 3000
====> Benchmarking: 7/3000   tot 3000
====> Benchmarking: 8/3000   tot 3000
====> Benchmarking: 9/3000   tot 3000
====> Benchmarking: 10/3000   tot 3000
====> Benchmarking: 11/3000   tot 3000
====> Benchmarking: 12/3000   tot 3000
====> Benchmarking: 13/3000   tot 3000
====> Benchmarking: 14/3000   tot 3000
====> Benchmarking: 15/3000   tot 3000
====> Benchmarking: 16/3000   tot 3000
====> Benchmarking: 17/3000   tot 3000
====> Benchmarking: 18/3000   tot 3000
====> Benchmarking: 19/3000   tot 3000
====> Benchmarking: 20/3000   tot 3000
====> Benchmarking: 21/3000   tot 3000
====> Benchmarking: 22/3000   tot 3000
====> Benchmarking: 23/3000   tot 3000
====> Benchmarking: 24/3000   tot 3000
====> Benchmarking: 25/3000   tot 3000
====> Benchmarking: 26/3000   tot 3000
====> Benchmarking: 27/3000   tot